<a href="https://colab.research.google.com/github/amitsiwach/CNN/blob/main/DenseNet%20%26%20EfficientNetB5/EfficientNetB5_on_Flower_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from glob import glob
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.densenet import preprocess_input
from keras.applications.efficientnet import preprocess_input

In [10]:
input_shape_efficientnet = (224, 224, 3)

from tensorflow.keras.applications import EfficientNetB5
efficient_model = EfficientNetB5(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape_efficientnet,
    pooling=None
)

In [11]:
# don't train existing weights
for layer in efficient_model.layers:
  layer.trainable = False

In [12]:
# useful for getting number of classes
folders = glob('/content/drive/MyDrive/Datasets/data for VGG Net/train/*')
print(len(folders))

4


In [13]:
#Give dataset path
train_path = '/content/drive/MyDrive/Datasets/data for VGG Net/train'
test_path = '/content/drive/MyDrive/Datasets/data for VGG Net/test'

In [14]:
x = Flatten()(efficient_model.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=efficient_model.input, outputs=prediction)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

In [15]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [16]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [17]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/data for VGG Net/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 475 images belonging to 4 classes.


In [18]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/data for VGG Net/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 264 images belonging to 4 classes.


In [19]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [20]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]


model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)

<ipython-input-20-3b8e16bac612>:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 3.6907 - accuracy: 0.6774


Epoch 1: val_loss improved from inf to 2.59552, saving model to mymodel.h5
5/5 [==============================] - 174s 35s/step - loss: 3.6907 - accuracy: 0.6774 - val_loss: 2.5955 - val_accuracy: 0.9091 - lr: 0.0100
Epoch 2/10
5/5 [==============================] - ETA: 0s - loss: 3.5666 - accuracy: 0.9226

5/5 [==============================] - 46s 8s/step - loss: 3.5666 - accuracy: 0.9226 - lr: 0.0100
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 3.2807 - accuracy: 0.9677

5/5 [==============================] - 45s 9s/step - loss: 3.2807 - accuracy: 0.9677 - lr: 0.0100
Epoch 4/10
5/5 [==============================] - ETA: 0s - loss: 2.5828 - accuracy: 0.9625

...

5/5 [==============================] - 39s 7s/step - loss: 2.7693 - accuracy: 0.9548 - lr: 0.0100
Epoch 7/10
5/5 [==============================] - ETA: 0s - loss: 1.5092 - accuracy: 0.9484

5/5 [==============================] - 43s 9s/step - loss: 1.5092 - accuracy: 0.9484 - lr: 0.0100
Epoch 8/10
5/5 [==============================] - ETA: 0s - loss: 3.5592 - accuracy: 0.9613

5/5 [==============================] - 41s 8s/step - loss: 3.5592 - accuracy: 0.9613 - lr: 0.0100
Epoch 9/10
5/5 [==============================] - ETA: 0s - loss: 0.8354 - accuracy: 0.9871

5/5 [==============================] - 39s 8s/step - loss: 0.8354 - accuracy: 0.9871 - lr: 0.0100
Epoch 10/10
5/5 [==============================] - ETA: 0s - loss: 1.2456 - accuracy: 0.9613

5/5 [==============================] - 39s 8s/step - loss: 1.2456 - accuracy: 0.9613 - lr: 0.0100
